In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_curve, auc
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
import sklearn.metrics as mt
from plotly import tools
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')



In [2]:
import pandas as pd

# Open the original CSV file for reading and a new CSV file for writing
with open('data.csv', 'r') as infile, open('cleaned.csv', 'w') as outfile:
    for line in infile:
        # Your data cleaning logic goes here
        # For example, you can check for and fix incomplete strings
        cleaned_line = line.replace('"incomplete data', '"complete data')

        # Write the cleaned line to the new file
        outfile.write(cleaned_line)

# Now you can read the cleaned data using pandas
df = pd.read_csv('cleaned.csv')


In [3]:
import pandas as pd

# Attempt to read the CSV file with error handling
try:
    df1 = pd.read_csv('data.csv', encoding='utf-8', error_bad_lines=False)
except pd.errors.ParserError as e:
    print(f"Error reading CSV file: {e}")
    df1 = None

if df1 is not None:
    # Select columns of interest
    df = df1[['Review Text', 'Rating', 'Class Name', 'Age']]
    df1.head()
else:
    print("CSV file couldn't be loaded due to parsing errors.")


In [4]:
df1.describe()

,Clothing ID,Age,Rating,Recommended IND,Positive Feedback Count
count,43486.000000,43486.000000,43486.000000,43486.000000,43486.000000
mean,795.547740,41.232075,3.645955,0.674470,5.976406
std,293.608318,12.503245,1.391089,0.468578,6.951038
min,0.000000,18.000000,1.000000,0.000000,0.000000
25%,610.000000,32.000000,3.000000,0.000000,0.000000
50%,867.000000,40.000000,4.000000,1.000000,3.000000
75%,1045.000000,51.000000,5.000000,1.000000,11.000000
max,1205.000000,99.000000,5.000000,1.000000,122.000000


In [6]:
df1.shape

(43487, 10)

In [5]:
df1.head()

,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,1082.0,18.0,Versatile Wonderful Cute Love Love Cute,Dress looks like it's made of,3.0,0.0,10.0,General,Jackets,Outerwear
1,827.0,45.0,"Loved, but returned Flattering Great Super cute",I'm torn! Stylish Love Wonderful,4.0,1.0,16.0,General,Dresses,Dresses
2,474.0,48.0,Not for the very petite,Super cute and cozy Great Stylish,5.0,1.0,9.0,General,Intimates,Outerwear
3,791.0,34.0,Love this Super cute Super cute Love Cute,My favorite buy Super cute Wonderful,1.0,0.0,13.0,General,Jackets,Pants
4,667.0,37.0,Flattering Great Cute Super cute Love Stylish,Love this Versatile Love Cute Love,4.0,1.0,11.0,General,Intimates,Lounge


In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense
from sklearn.metrics import classification_report
from tensorflow.keras.layers import Dropout

df = pd.read_csv('cleaned.csv')  # If you're reading from a CSV file

# Check the data types of the DataFrame
print(df.dtypes)

# If there are any float columns that should be treated as strings, convert them
# For example, if 'Review Text' is a float, you might need to convert it to string
df['Review Text'] = df['Review Text'].astype(str)

# Now you can proceed with tokenization
tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(df['Review Text'])
sequences = tokenizer.texts_to_sequences(df['Review Text'])

# Pad sequences to ensure uniform input size
padded_sequences = pad_sequences(sequences, maxlen=100, padding='post', truncating='post')

# Convert sentiment labels to binary (0 for negative, 1 for positive)
# Assuming 'Rating' is the column to be used for sentiment analysis
df['Sentiment'] = df['Rating'].apply(lambda x: 1 if x >= 4 else 0)
labels = df['Sentiment'].astype(int)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)

# Define the CNN model
model = Sequential([
    Embedding(5000, 100, input_length=100),
    Conv1D(32, 5, activation='relu'),
    Dropout(0.5),  # Add dropout after the Conv1D layer
    GlobalMaxPooling1D(),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {accuracy * 100:.2f}%")

# Predict the classes for the test set
y_pred = (model.predict(X_test) > 0.5).astype("int32")

# Print the classification report
print(classification_report(y_test, y_pred))

Clothing ID                float64
Age                        float64
Title                       object
Review Text                 object
Rating                     float64
Recommended IND            float64
Positive Feedback Count    float64
Division Name               object
Department Name             object
Class Name                  object
dtype: object
Epoch 1/10
1088/1088 [==============================] - 27s 24ms/step - loss: 0.5182 - accuracy: 0.7178 - val_loss: 0.4956 - val_accuracy: 0.7379
Epoch 2/10
1088/1088 [==============================] - 28s 26ms/step - loss: 0.4537 - accuracy: 0.7545 - val_loss: 0.4874 - val_accuracy: 0.7494
Epoch 3/10
1088/1088 [==============================] - 31s 29ms/step - loss: 0.4261 - accuracy: 0.7688 - val_loss: 0.4835 - val_accuracy: 0.7526
Epoch 4/10
1088/1088 [==============================] - 27s 25ms/step - loss: 0.4034 - accuracy: 0.7798 - val_loss: 0.4815 - val_accuracy: 0.7483
Epoch 5/10
1088/1088 [==============================